In [1]:
using LinearAlgebra
using Random
using StatsBase
using GraphPlot
using Graphs
using DelimitedFiles
using LaTeXStrings
using CSV,DataFrames

In [2]:
data_set_name = ["Karate", "webkb", "ppi", "dd199"]
X = data_set_name[1]
edge_data = CSV.read("dataset/"*"$X"*"/"*"$X"*"_edge.csv", header=0, delim=" ", DataFrame)

,Column1,Column2
,Int64,Int64
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,11,1


In [3]:
function Data_network(data)
    n = maximum([maximum(data.Column1),maximum(data.Column2)])
    g = Graphs.SimpleGraph(n)
    len = length(data[:,1])
    for i in 1:len
        Graphs.add_edge!(g, data[i,1], data[i,2])
        Graphs.add_edge!(g, data[i,2], data[i,1])
    end
    return g
end
#function main_component(g)
#    c = connected_components(g)
#    _, i = findmax(length.(c))
#    g[c[i]]
#end
G = Data_network(edge_data)

{34, 78} undirected simple Int64 graph

# determining param $w_{i,j}^{v_0}$

In [16]:
function w_ij_x0(l, i, j, p, q)    
    if l[i] == 0
        return 1.0
    elseif l[i] > l[j]
        return 1/p
    elseif l[i] == l[j]
        return 1/q
    elseif l[i] < l[j]
        return 1/q
    end
end

function determin_param(G, v0, p, q)
    param_list = Dict{Tuple{Int, Int}, Float64}()
    dijkstra_result = Graphs.dijkstra_shortest_paths(G, v0)
    l = dijkstra_result.dists
    neighbors = [Graphs.all_neighbors(G, i) for i in 1:Graphs.nv(G)]
    
    for i in 1:Graphs.nv(G)
        for j in neighbors[i]
            param_list[(i,j)] = w_ij_x0(l, i, j, p, q)    
        end
    end
    return param_list
end
p = 0.5
q = 0.1
v0 = 2
#println("1/w_p=",1/p)
#println("1/w_q=",1/q)
data1 = determin_param(G, v0, p, q)

Dict{Tuple{Int64, Int64}, Float64} with 156 entries:
  (31, 33) => 10.0
  (1, 12)  => 10.0
  (3, 29)  => 10.0
  (7, 1)   => 2.0
  (28, 34) => 10.0
  (33, 23) => 10.0
  (23, 34) => 2.0
  (3, 28)  => 10.0
  (7, 17)  => 10.0
  (9, 3)   => 2.0
  (25, 26) => 10.0
  (2, 22)  => 1.0
  (5, 11)  => 10.0
  (1, 9)   => 10.0
  (2, 1)   => 1.0
  (21, 33) => 2.0
  (18, 1)  => 10.0
  (29, 3)  => 2.0
  (33, 15) => 10.0
  (34, 10) => 10.0
  (2, 18)  => 1.0
  (2, 8)   => 1.0
  (34, 27) => 10.0
  (34, 24) => 10.0
  (6, 11)  => 10.0
  ⋮        => ⋮

In [17]:
function dict_neighbor_list(G)
    dict = Dict()
    for node in 1:Graphs.nv(G)
        dict[node] = Graphs.all_neighbors(G, node)
    end
    return dict
end
#dict_neighbor_list(G)

function coin_dict(G, v0, p, q)
    N = Graphs.nv(G)
    dict_Ci = Dict()
    determin_results = Dict()
    weight_param = determin_param(G, v0, p, q)
    neighbor_dict = dict_neighbor_list(G)
    for node in 1:N
        neighbor_list = neighbor_dict[node]
        k = length(neighbor_list)
        array = ones(Float64, k)
        
        for (idx, j) in enumerate(neighbor_list)
            if !haskey(determin_results, (node, j))
                determin_results[(node, j)] = weight_param[(node, j)]
            end
            array[idx] = determin_results[(node, j)]
        end
        
        ψ = sqrt.(array) ./ sqrt(sum(array))
        dict_Ci[node] = 2 * ψ * ψ' - Matrix{Float64}(I, k, k)
    end
    return dict_Ci
end

function ψ_dict(G, v0, p, q)
    N = Graphs.nv(G)
    dict_ψ0 = Dict()
    dict_ψ1 = Dict{Tuple{Int, Int}, Float64}()
    determin_results = Dict()
    weight_param = determin_param(G, v0, p, q)
    neighbor_dict = dict_neighbor_list(G)
    for node in 1:N
        neighbor_list = neighbor_dict[node]
        k = length(neighbor_list)
        array = ones(Float64, k)
        
        for (idx, j) in enumerate(neighbor_list)
            if !haskey(determin_results, (node, j))
                determin_results[(node, j)] = weight_param[(node, j)]
            end
            array[idx] = determin_results[(node, j)]
        end
        ##initial state
        ψ = sqrt.(array) ./ sqrt(sum(array) * N)
        dict_ψ0[node] = ψ
    end
    
    for node in 1:N
        neighbor_list = neighbor_dict[node]
        for (idx, j) in enumerate(neighbor_list)
            dict_ψ1[(node, j)] = dict_ψ0[node][idx]
        end
    end
    return dict_ψ1
end


ψ_dict (generic function with 1 method)

In [18]:
function DTQW(G,v0,p,q,itr)
    N = Graphs.nv(G)
    weight_param = determin_param(G, v0, p, q)
    neighbor_dict = dict_neighbor_list(G)
    coin = coin_dict(G, v0, p, q)
    dict_ψt = ψ_dict(G,v0,p,q)
    feature_vec_v0 = zeros(N,itr)
    
    for t in 1:itr
        next_dict_ψt = Dict{Tuple{Int, Int}, Float64}()
        for node in 1:N
            neighbor_list = neighbor_dict[node]
            k = length(neighbor_list)
            array = ones(Float64, k)
            for (idx, j) in enumerate(neighbor_list)
                array[idx] = dict_ψt[(node,j)]
            end
            Cψ = coin[node]*array
            
            #shift operator
            for (idx,j) in enumerate(neighbor_list)
                next_dict_ψt[(j,node)] = Cψ[idx]
            end
        end
        dict_ψt = next_dict_ψt
        
        #prob
        dict_prob = Dict{Int, Float64}()
        for node in 1:N
            neighbor_list = neighbor_dict[node]
            k = length(neighbor_list)
            array = ones(Float64, k)
            for (idx, j) in enumerate(neighbor_list)
                #println(node, j)
                array[idx]　= dict_ψt[(node,j)]^2
            end
            dict_prob[node] = sum(array)
        end
        
        for node in 1:N
            feature_vec_v0[node,t] = dict_prob[node]
        end
        
    end
    return feature_vec_v0
end

DTQW (generic function with 1 method)

In [19]:
using ProgressBars

function feature_vector(G,p,q,itr)
    N = Graphs.nv(G)
    feature_vec = zeros(N,itr)
    for v0 in 1:N
        feature_vec .+= DTQW(G,v0,p,q,itr)
    end
    return feature_vec
end

feature_vector (generic function with 1 method)

In [20]:
itr = 400
param_list = [0.25,0.50,1.0,2.0,4.0]


function make_dataset(itr,param_list)
    for p in tqdm(param_list)
        for q in param_list
            feature_vec = feature_vector(G,p,q,itr)
            dir_name = "feature_vec_of_qw/"*"$X"*"/superposition/modify"
            file_name = "/p$p"*"q$q"*"_feature_vec_itr$itr.csv"
            #dir_name = "feature_vec_of_qw/"*"$X"*"/superposition/"*"p$p"*"q$q"*"_feature_vec_itr$itr.csv"
            #mkdir(dir_name)
            writedlm( dir_name*file_name,  feature_vec, ',')
        end
    end
end

make_dataset (generic function with 1 method)

In [21]:
make_dataset(itr,param_list)

0.0%┣                                                ┫ 0/5 [00:00<00:00, -0s/it]
20.0%┣████████▋                                  ┫ 1/5 [00:09<Inf:Inf, InfGs/it]
40.0%┣██████████████████▉                            ┫ 2/5 [00:18<00:55, 18s/it]
60.0%┣████████████████████████████▏                  ┫ 3/5 [00:27<00:27, 14s/it]
80.0%┣█████████████████████████████████████▋         ┫ 4/5 [00:36<00:12, 12s/it]
100.0%┣██████████████████████████████████████████████┫ 5/5 [00:45<00:00, 11s/it]
100.0%┣██████████████████████████████████████████████┫ 5/5 [00:45<00:00, 11s/it]
